In [1]:
import requests
from newspaper import Article

# -------- CONFIG --------
SERPER_API_KEY = "b6377b6d2d07fe6a6f334f03986986351846d4bb"  # 🔁 Replace with your actual key
OLLAMA_API_URL = "http://localhost:11434/api/generate"

In [2]:
import re 

In [3]:
pip install playwright


Note: you may need to restart the kernel to use updated packages.


In [4]:
playwright install


SyntaxError: invalid syntax (2310654265.py, line 1)

In [ ]:
!pip install selenium


In [5]:
def extract_article_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return f"Source: {url}\n\n{article.text}"
    except Exception as e:
        return f"Source: {url}\n\n[Failed to extract: {e}]"

In [6]:
def ask_llama3_locally(prompt):
    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": prompt, "stream": False},
            headers=headers,
            verify=False  # Disable SSL verification for ngrok URL
        )
        if response.status_code == 200:
                answer_text = response.json().get("response", "[No response found]")
                return answer_text  # ✅ Return only the clean text
        else:
            return f"[HTTP {response.status_code}] Error from Llama3"
    except Exception as e:
        return f"[❌ Error calling Llama3: {e}]"

In [7]:
from bs4 import BeautifulSoup

def clean_html(html):
    soup = BeautifulSoup(html, "html.parser")
    texts = soup.find_all(text=True)

    blacklist = [
        '[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', 'style'
    ]

    output = ''
    for t in texts:
        if t.parent.name not in blacklist:
            content = t.strip()
            if content:
                output += f'{content} '

    return output


In [8]:
def is_url_trusted_by_ai(url):
    prompt = f"""You are a trustworthiness evaluator.

Decide whether the following website is generally considered a **professional and trustworthy source of information** for serious research, financial news, or due diligence.

Do **not** include websites that:
- Are editable by the public (like Wikipedia, Reddit, Quora)
- Are known for unverified or user-generated content

Only reply with `yes` or `no`.

### Website:
{url}

### Is this source trusted?
"""

    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": prompt, "stream": False},
            headers=headers,
            verify=False
        )
        if response.status_code == 200:
            verdict = response.json().get("response", "").strip().lower()
            print(f"🤖 AI verdict for {url}: {verdict}")
            return "yes" in verdict
        else:
            return False
    except Exception as e:
        print(f"[AI trust check failed for {url}]: {e}")
        return False


In [9]:
def web_search_serper(query, max_results=4):
    headers = {
        "X-API-KEY": SERPER_API_KEY
    }
    json_data = {
        "q": query
    }
    response = requests.post("https://google.serper.dev/search", headers=headers, json=json_data)

    if response.status_code != 200:
        print(f"❌ Serper API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("organic", [])
    urls = []
    for item in results:
        url = item["link"]
        print(f"🔍 Evaluating source: {url}")
        if is_url_trusted_by_ai(url):
            urls.append(url)
        if len(urls) >= max_results:
            break
    return urls


In [10]:
def classify_question_type(query):
    classification_prompt = f"""You are a classification assistant.

Decide if the following question requires a short, direct answer (like a number, name, or fact), or a detailed, multi-paragraph summary.

Respond with only one word: `short` or `detailed`.

### Question:
{query}

### Answer Style:"""

    # Send to LLaMA3
    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": classification_prompt, "stream": False},
            headers=headers,
            verify=False
        )
        if response.status_code == 200:
            style = response.json().get("response", "").strip().lower()
            return "short" if "short" in style else "detailed"
        else:
            return "detailed"
    except:
        return "detailed"


In [11]:
def display_final_answer(answer, query, urls):
    print("tetete"+answer)
    print("\n" + "="*80)
    print(f"🧠 Answer to: {query}")
    print("="*80)
    print("\n🔗 Sources Used:")
    for url in urls:
        print(f" - {url}")
    print("="*80)


In [ ]:
from playwright.async_api import async_playwright
import requests
from bs4 import BeautifulSoup

async def extract_readable_text1(url):
    try:
        # Try normal requests first
        response = requests.get(url, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator="\n").strip()

        if len(text) < 500:
            print(f"🔄 Switching to Async Playwright for {url}...")






            
            return await extract_with_playwright_async(url)

        return text

    except Exception as e:
        print(f"❌ Error using requests on {url}: {e}")
        print(f"🔄 Switching to Async Playwright for {url}...")
        return await extract_with_playwright_async(url)


async def extract_with_playwright_async(url):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.goto(url, timeout=60000)
            await page.wait_for_load_state('networkidle')

            html = await page.content()
            await browser.close()

            soup = BeautifulSoup(html, "html.parser")
            text = soup.get_text(separator="\n").strip()
            return text
    except Exception as e:
        print(f"❌ Playwright async extraction failed for {url}: {e}")
        return ""


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def extract_readable_text2(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    )
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                  get: () => undefined
                })
            """
        }
    )

    driver.get(url)
    time.sleep(15)  # Enough for heavy JS
    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    # 🚀 Only focus on main content tags
    whitelist = ['p', 'h1', 'h2', 'h3', 'h4', 'li', 'span', 'a', 'td', 'th', 'strong', 'b']

    output = ''
    for tag in soup.find_all(whitelist):
        text = tag.get_text(strip=True)
        if text and len(text) > 5:  # Ignore too short junk
            output += f"{text}\n"

    return output

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time


def extract_readable_text(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=chrome_options)

    # Make webdriver harder to detect
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        """
    })

    driver.get(url)
    time.sleep(3)  # Let the page start rendering

    # Scroll slowly to trigger lazy loading elements
    scroll_pause_time = 1.0
    last_height = driver.execute_script("return document.body.scrollHeight")

    for _ in range(3):  # Scroll down 3 times
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(2)  # Wait a little extra to be sure content is loaded

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    texts = soup.find_all(string=True)

    output = ''
    blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', 'style', 'iframe', 'svg']

    for t in texts:
        if t.parent.name not in blacklist and t.strip():
            output += '{} '.format(t.strip())

    return output


# Example usage:
if __name__ == "__main__":
    url = "https://cointelegraph.com/tags/ethereum"
    text = extract_readable_text(url)
    print(text[:1000])  # Show first 1000 characters

Ethereum News Today & ETH Predictions | Cointelegraph Ecosystem English News Indices In Depth Learn Newsletters Crypto Bonus Research Podcasts About BTC $94,025 0.46% ETH $1,783 0.21% XRP $2.18 0.66% BNB $601.75 0.55% SOL $148.08 4.25% DOGE $0.181 0.87% ADA $0.7056 1.92% STETH $1,780.71 0.04% TRX $0.251 3.13% AVAX $21.88 3.04% SUI $3.41 8.20% TON $3.21 0.42% BTC $94,013 0.48% ETH $1,782 0.25% XRP $2.18 0.66% BNB $601.68 0.56% SOL $148.10 4.07% DOGE $0.1811 0.69% ADA $0.7058 1.72% STETH $1,780.71 0.04% TRX $0.251 3.13% AVAX $21.90 2.95% SUI $3.42 8.14% TON $3.21 0.36% BTC $94,025 0.46% ETH $1,783 0.20% XRP $2.18 0.62% BNB $601.68 0.56% SOL $148.10 4.07% DOGE $0.1811 0.69% ADA $0.7058 1.72% STETH $1,780.71 0.04% TRX $0.251 3.13% AVAX $21.90 2.95% SUI $3.42 8.14% TON $3.21 0.36% USD Ad Ethereum (ETH) Crypto News Today Ad Win prizes Crypto assets are a high-risk investment. You should consider whether you understand the possibility of losing money due to leverage. None of the material shou

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import time

def extract_readable_text1(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--start-maximized")

    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(options=chrome_options)

    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {get: () => undefined})
            """
        }
    )

    driver.get(url)

    # Let JS heavy pages load
    time.sleep(8)  # 8s is a good compromise

    try:
        # Get all visible text
        visible_texts = driver.execute_script("""
            let textNodes = [];
            function getVisibleTextNodes(node) {
                if (node.nodeType === Node.TEXT_NODE && node.nodeValue.trim() !== '') {
                    const style = window.getComputedStyle(node.parentElement);
                    if (style && style.visibility !== 'hidden' && style.display !== 'none') {
                        textNodes.push(node.nodeValue.trim());
                    }
                }
                for (let child of node.childNodes) {
                    getVisibleTextNodes(child);
                }
            }
            getVisibleTextNodes(document.body);
            return textNodes;
        """)

        full_text = "\n".join(visible_texts)

    except Exception as e:
        print(f"❌ Error extracting text: {e}")
        full_text = ""

    driver.quit()

    return full_text

In [13]:
import re

def smart_clean_text(text):
    lines = text.split("\n")
    important_lines = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Filter rules:
        if "cookie" in line.lower():
            continue
        if "privacy" in line.lower():
            continue
        if "consent" in line.lower():
            continue
        if "sign up" in line.lower() or "sign in" in line.lower():
            continue
        if "accept" in line.lower() and "cookies" in line.lower():
            continue
        if len(line) < 5:
            continue
        important_lines.append(line)

    cleaned_text = "\n".join(important_lines)
    return cleaned_text

# Usage

# Then build prompt


In [23]:
def web_rag_query(query):
    print(f"🔍 Searching with Serper for: {query}")
    urls = web_search_serper(query)

    # --- Blacklist bad URLs ---
    blacklist = [
        "https://bitflyer.com/en-us/bitcoin-chart"
    ]

    # Remove blacklisted URLs
    urls = [url for url in urls if url not in blacklist]

    print(f"📎 URLs after blacklist filter: {urls}")
    if not urls:
        return "[❌ No articles found for this query.]", urls

    print(f"📄 Extracting and summarizing content from {len(urls)} URLs...")
    summaries = []
    successful_urls = []

    for url in urls:
        try:
            raw_text = extract_readable_text(url)

            # 🧠 Now create a query-focused summarization prompt
            summarize_prompt = f"""
User question: {query}

Summarize only the parts of this page that are relevant to answering the user's question.

Page Content:
{raw_text}
"""
            summary = ask_llama3_locally(summarize_prompt)

            source_text = f"Source: {url}\n---\n{summary.strip()}"
            summaries.append(source_text)
            print(f"✅ Successfully summarized content from {url}")
            successful_urls.append(url)

        except Exception as e:
            print(f"❌ Failed to extract or summarize {url}: {e}")
            continue

    if not summaries:
        return "[❌ No accessible or relevant sources found.]", urls

    joined_summaries = "\n\n".join(summaries)

    answer_style = classify_question_type(query)

    if answer_style == "short":
        final_prompt = f"""You are a precise assistant. Give a short, factual answer based **only** on the provided sources.

### Question:
{query}

### Sources:
{joined_summaries}

### Answer:
"""
    else:
        final_prompt = f"""You are an expert crypto analyst and news summarizer.

Please answer the following question using only the information from the provided sources.
Be clear, concise, and neutral in tone.

### Question:
{query}

### Sources:
{joined_summaries}

### Summary:
"""

    print("🧠 Asking llama3 locally...")
    answer = ask_llama3_locally(final_prompt)
    return answer, successful_urls

In [ ]:
def web_rag_query1(query):
    
    print(f"🔍 Searching with Serper for: {query}")
    urls = web_search_serper(query)

    # --- Blacklist bad URLs ---
    blacklist = [
        "https://bitflyer.com/en-us/bitcoin-chart"
    ]


    # Remove blacklisted URLs
    urls = [url for url in urls if url not in blacklist]

    print(f"📎 URLs after blacklist filter: {urls}")
    if not urls:
        return "[❌ No articles found for this query.]", urls

    print(f"📄 Extracting content from {len(urls)} URLs...")
    sources = []
    successful_urls = []

    for url in urls:
        try:
            raw_text =  extract_readable_text(url)
            
            # 👇 Separate each source properly
            source_text = f"Source: {url}\n---\n{raw_text}\n" 
            sources.append(source_text)
            print(f"✅ Extracted from {sources}")
            successful_urls.append(url)
          
        except Exception as e:
            print(f"❌ Failed to extract {url}: {e}")
            continue

    if not sources:
        return "[❌ No accessible or relevant sources found.]", urls

    answer_style = classify_question_type(query)

    # 👇 Here, join sources with two newlines between them for clarity
    joined_sources = "\n\n".join(sources)

    if answer_style == "short":
        final_prompt = f"""You are a precise assistant. Give a short, factual answer based **only** on the provided sources.

### Question:
{query}

### Sources:
{joined_sources}

### Answer:
"""
    else:
        final_prompt = f"""You are an expert crypto analyst and news summarizer.

Please answer the following question using only the information from the provided sources.
Be clear, concise, and neutral in tone.

### Question:
{query}

### Sources:
{joined_sources}

### Summary:
"""

    print("🧠 Asking llama3 locally...")
    answer = ask_llama3_locally(final_prompt)
    return answer, successful_urls


In [ ]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [25]:
query= "what is the price of bitcoin today"
final_answer, urls =  web_rag_query(query)
print("tetete"+final_answer)

🔍 Searching with Serper for: what is the price of bitcoin today
🔍 Evaluating source: https://coinmarketcap.com/currencies/bitcoin/
🤖 AI verdict for https://coinmarketcap.com/currencies/bitcoin/: **no**
🔍 Evaluating source: https://www.coindesk.com/price/bitcoin
🤖 AI verdict for https://www.coindesk.com/price/bitcoin: **yes**
🔍 Evaluating source: https://bitflyer.com/en-us/bitcoin-chart
🤖 AI verdict for https://bitflyer.com/en-us/bitcoin-chart: yes
🔍 Evaluating source: https://www.coinbase.com/price/bitcoin
🤖 AI verdict for https://www.coinbase.com/price/bitcoin: no
🔍 Evaluating source: https://ca.finance.yahoo.com/quote/BTC-USD/
🤖 AI verdict for https://ca.finance.yahoo.com/quote/BTC-USD/: yes
🔍 Evaluating source: https://www.binance.com/en/price/bitcoin
🤖 AI verdict for https://www.binance.com/en/price/bitcoin: no
🔍 Evaluating source: https://robinhood.com/us/en/crypto/BTC/
🤖 AI verdict for https://robinhood.com/us/en/crypto/BTC/: **no**
🔍 Evaluating source: https://www.tradingview.co